# Tutorial for HBMCMC code

The set of modules in acrg_hbmcmc allow a convenient way to perform 'hierachical Bayesian MCMC'. \
That is, we want to infer emissions of a partiular species (and the influence from the domain boundary) using a Markov Chain Monte Carlo algorithm. The 'hierarchy' is that this depends on 'hyperparameters', which are generally the uncertainties in the non-latent parameters involved in the estimation (i.e. not those we are trying to derive – the emissions – but others that are necessary, e.g. the measurement error). \
For a work-in-progress introuction to inverse modelling see: https://www.overleaf.com/project/5f8d6217aeca1900019a84ce 

The code within acrg_hbmcmc relies heavily on other code in the ACRG repository, mainly name.py in acrg_name. This code is generally used to read footprints, a priori emissions etc from the ACRG directory structure. \
Currently the MCMC estimation is completed using the pymc3 library (see https://docs.pymc.io/), a well-established statistical library.

## Running the code

The easiest way to run the hbmcmc code is to copy the file hbmcmc_input_template.ini in acrg_hbmcmc/config/ to your run directory, and edit this code with your desired set up. \
The code generally explains the various inputs, but we will go through them below:

**species** is a single string of the species you wish to do an inversion for, e.g. "CH4" or "CFC-11". As it says above, checkout out acrg_species_info.json for the various options currently available, or add your own if needed. \
**sites** is a list containing the sites you wish to use measurements from defined by their 3-letter code, e.g. ["MHD", "TAC"] for Mace Head and Tacolneston measurement sites. Again, as above, see acrg_site_info.json for options or add your own. \
**meas_period** is a list of the averaging you wish to apply to the measurements at the differnt sites. Often not much is gained by having many measurements made at really high frequency, especially as the footprints are rarely such high resolution. Instead we may wish to use a coarser frequency, e.g. every 6 hours, where all measurements made within a 6-hour period are averaged into a single measurement. A meas_period much be supplied in the list for each site, e.g. ["6H","6H"]. \
**start_date** is when you want the inversion to begin, as a string (see above). \
**end_date** is when you want the inversion to end, as a string (see above). Note that the day is not included (e.g. "2000-01-01" would be until 23:59 on 1999-12-31) \



In [ ]:
; inlet (list/None) - Specific inlet height for the site (list - must match number of sites)
; instrument (list/None) - Specific instrument for the site (list - must match number of sites)
; obs_directory (str/None) - Directory containing the obs data (with site codes as subdirectories)

inlet = None
instrument = None
obs_directory = None

[INPUT.PRIORS]
; Input values for extracting footprints, emissions and boundary conditions files (also uses values from INPUT.MEASUREMENTS)
; domain (str) - Name of inversion spatial domain e.g. "EUROPE"
; fpheight (dict/None) - Release height for footprints. e.g. fpheight = {"TAC":"185m"} (must match number of sites).
; emissions_name (dict/None) - Name for specific emissions source. this should be a dictionary with {source_name: emissions_file_identifier} (e.g. {'anth':'co2-ff-mth'})
; fp_directory (str/None) - Directory containing the footprints files (with domain name as subdirectories)
; flux_directory (str/None) - Directory containing the emissions files (with domain name as subdirectories)

domain = ''           ; (required) 
fpheight = None
emissions_name = None
fp_directory = None
flux_directory = None

[INPUT.BASIS_CASE]
; Input values to extract the basis cases to use within the inversion for boundary conditions and emissions
; bc_basis_case (str) - boundary conditions basis, defaults to "NESW" (looks for file format {bc_basis_case}_{domain}_*.nc)
; fp_basis_case (str/None) - emissions bases:
; - if specified, looks for file format {fp_basis_case}_{domain}_*.nc
; - if None, creates basis function using quadtree algorithm and associated parameters
;   - nbasis - Number of basis functions to use for quadtree derived basis function (rounded to %4)
; basis_directory (str/None) - Directory containing the basis functions (with domain name as subdirectories)

bc_basis_case = "NESW"

fp_basis_case = None
quadtree_basis = True
nbasis = 100
basis_directory = None

[MCMC.TYPE]
; Which MCMC setup to use. This defines the function which will be called and the expected inputs.
; Options include:
; "fixed_basis"

mcmc_type = "fixed_basis"

[MCMC.PDF]
; Definitions of PDF shape and parameters for inputs
; - xprior (dict) - emissions
; - bcprior (dict) - boundary conditions
; - sigprior (dict) - model error

; Each of these inputs should be dictionary with the name of probability distribution and shape parameters.
; See https://docs.pymc.io/api/distributions/continuous.html
; Current options for the "pdf" parameter include:

; - "lognormal" - Log-normal log-likelihood.
;  - "mu" (float) - Location parameter
;  - "sd" or "sigma" (float) - Standard deviation (> 0)
; e.g. {"pdf":"lognormal", "mu":1, "sd":1}

; - "uniform" - Continuous uniform log-likelihood.
;  - "lower" (float) - Lower limit
;  - "upper" (float) - Upper limit
; e.g. {"pdf":"uniform", "lower":0.5, "upper":3}

; - "halfflat" - Improper flat prior over the positive reals. (no additional parameters necessary)
; e.g. {"pdf":"halfflat"}


xprior = {"pdf":"lognormal", "mu":1, "sd":1}
bcprior = {"pdf":"lognormal", "mu":0.004, "sd":0.02}
sigprior = {"pdf":"uniform", "lower":0.5, "upper":3}


[MCMC.BC_SPLIT]
; Boundary conditions setup
; - bc_freq - The period over which the baseline is estimated. e.g.
;  - None - one scaling for the whole inversion
;  - "monthly" - per calendar monthly
;  - "*D" (e.g. "30D") - per number of days (e.g. 30 days)

bc_freq = None
sigma_freq = None
sigma_per_site = True

[MCMC.ITERATIONS]
; Iteration parameters
; nit (int) - Number of iterations for MCMC
; burn (int) - Number of iterations to burn in MCMC
; tune (int) - Number of iterations to use to tune step size

nit = 2.5e5
burn = 50000
tune = 1.25e5

[MCMC.NCHAIN]
; Number of chains to run simultaneously. Must be >=2 to allow convergence to be checked.

nchain = 2

[MCMC.ADD_ERROR]
; Add variability in averaging period to the measurement error

averagingerror = True

[MCMC.OUTPUT]
; Details of where to write the output
; outputpath (str) - directory to write output
; outputname (str) - unique identifier for output/run name.

outputpath = ''  ; (required)
outputname = ''  ; (required)

